## Dogs vs Cats Image Classification(without image augmentation)

### Import Packages

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import matplotlib.pyplot as plt
import numpy as np

2023-10-06 15:04:54.857343: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-06 15:04:55.174027: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-06 15:04:55.175556: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-06 15:05:07.027953: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

### Data Loading

#### Download the datasets and create the path for training and validation

In [3]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
zip_dir = tf.keras.utils.get_file('cats_and_dogs_filterted.zip', origin=_URL, extract=True)

zip_dir_base = os.path.dirname(zip_dir)
!find $zip_dir_base -type d -print

/home/login/.keras/datasets
/home/login/.keras/datasets/cats_and_dogs_filtered
/home/login/.keras/datasets/cats_and_dogs_filtered/validation
/home/login/.keras/datasets/cats_and_dogs_filtered/validation/dogs
/home/login/.keras/datasets/cats_and_dogs_filtered/validation/cats
/home/login/.keras/datasets/cats_and_dogs_filtered/train
/home/login/.keras/datasets/cats_and_dogs_filtered/train/dogs
/home/login/.keras/datasets/cats_and_dogs_filtered/train/cats


In [4]:
base_dir = os.path.join(os.path.dirname(zip_dir), 'cats_and_dogs_filtered')
print(base_dir)
train_dir = os.path.join(base_dir, 'train')
print(train_dir)
validation_dir = os.path.join(base_dir, 'validation')


train_cats_dir = os.path.join(train_dir, 'cats')
print(train_cats_dir)
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
print(validation_dogs_dir)

/home/login/.keras/datasets/cats_and_dogs_filtered
/home/login/.keras/datasets/cats_and_dogs_filtered/train
/home/login/.keras/datasets/cats_and_dogs_filtered/train/cats
/home/login/.keras/datasets/cats_and_dogs_filtered/validation/dogs


#### Trying to access a manually downloaded dataset from my local files

In [6]:
path = '/home/login/Documents/Intro_to_TF/cats_and_dogs_filtered/'
!find $path -type d -print

/home/login/Documents/Intro_to_TF/cats_and_dogs_filtered/
/home/login/Documents/Intro_to_TF/cats_and_dogs_filtered/validation
/home/login/Documents/Intro_to_TF/cats_and_dogs_filtered/validation/dogs
/home/login/Documents/Intro_to_TF/cats_and_dogs_filtered/validation/cats
/home/login/Documents/Intro_to_TF/cats_and_dogs_filtered/train
/home/login/Documents/Intro_to_TF/cats_and_dogs_filtered/train/dogs
/home/login/Documents/Intro_to_TF/cats_and_dogs_filtered/train/cats


In [7]:
# Define the training dataset paths
train_path = os.path.join(path, "train")
print(train_path)
train_cats_path = os.path.join(train_path, "cats")
train_dogs_path = os.path.join(train_path, "dogs")

# Define the validation dataset paths
validation_path = os.path.join(path, "validation")
print(validation_path)
validation_cats_path = os.path.join(validation_path, "cats")
validation_dogs_path = os.path.join(validation_path, "dogs")

/home/login/Documents/Intro_to_TF/cats_and_dogs_filtered/train
/home/login/Documents/Intro_to_TF/cats_and_dogs_filtered/validation


### Understanding our data

In [9]:
# Sample data
os.listdir(train_cats_path)[32:35]

['cat.760.jpg', 'cat.103.jpg', 'cat.127.jpg']

In [10]:
num_cats_train = len(os.listdir(train_cats_path))
print(f"Number of trainng cats images: {num_cats_train}")
num_dogs_train = len(os.listdir(train_dogs_path))
print(f"Number of training dogs images: {num_dogs_train}", end = "\n\n")

num_cats_val = len(os.listdir(validation_cats_path))
print(f"Number of validation cats images: {num_cats_val}")
num_dogs_val = len(os.listdir(validation_dogs_path))
print(f"Number of validation dogs images: {num_dogs_val}", end = "\n\n")

print(f"Total training data: {num_cats_train + num_dogs_train}")
print(f"Total validation data: {num_cats_val + num_dogs_val}")

Number of trainng cats images: 1000
Number of training dogs images: 1000

Number of validation cats images: 500
Number of validation dogs images: 500

Total training data: 2000
Total validation data: 1000


In [11]:
train_cats_path

'/home/login/Documents/Intro_to_TF/cats_and_dogs_filtered/train/cats'

#### Setting model parameters

In [13]:
BATCH_SIZE = 100
IMAGE_SHAPE = 150

### Data Preparation

In [14]:
train_image_generator = ImageDataGenerator(rescale = 1./255)
validation_image_generator = ImageDataGenerator(rescale = 1./255)